In [34]:
import nltk
ss = nltk.corpus.brown.tagged_sents(tagset='universal')
ww = nltk.corpus.brown.tagged_words(tagset='universal')

In [57]:
from collections import defaultdict
sent_len = defaultdict(int)
for s in ss:
    sent_len[len(s)] += 1

nltk.corpus.reader.util.ConcatenatedCorpusView

In [9]:
import numpy as np

In [10]:
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/tianhaolu/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [11]:
wd_count = 0
wd_idx = {}
tag_count = 0
tag_idx = {}

In [12]:
for pair in ww:
    wd, tag = pair[0], pair[1]
    if wd not in wd_idx:
        wd_idx[wd] = wd_count
        wd_count += 1
    if tag not in tag_idx:
        tag_idx[tag] = tag_count
        tag_count += 1
    

In [46]:
tag_count

12

In [13]:
tag_idx

{'.': 5,
 'ADJ': 2,
 'ADP': 4,
 'ADV': 6,
 'CONJ': 7,
 'DET': 0,
 'NOUN': 1,
 'NUM': 10,
 'PRON': 9,
 'PRT': 8,
 'VERB': 3,
 'X': 11}

goal is to maximize <br>
$l(\theta) = \Sigma_{i}^{N}logp(y^i|x^i)$
<br>

$ = \Sigma_{i}^{N}\Sigma_{t}^{T}\Sigma_{k}^{K}\lambda_k f_k(y_t^i,y_{t-1}^i, x_t^i) - \Sigma_{i}^{N}logZ(x^i)$
<br>
With Gradient Ascent, we need to compute
<br>
<br>
$\frac{\partial l}{\partial \lambda_k} = \Sigma_{i}^{N}\Sigma_{t}^{T}f_k(y_t^i,y_{t-1}^i, x_t^i) - \Sigma_{i}^{N}\Sigma_{t}^{T}\Sigma_{y, y'}f_k(y,y',x_t^i)p(y,y'|x^i)$

Assume M tag states and V vocabs, we need M*M+M*V feature fns;

The first term $\Sigma_{i}^{N}\Sigma_{t}^{T}f_k(y_t^i,y_{t-1}^i, x_t^i)$ can be cached as it remains the same w.r.t. empirical (training) data

In [58]:
emp_sum_yy = np.zeros((tag_count, tag_count))
emp_sum_yx = np.zeros((tag_count, wd_count))

In [59]:
for i, sent in enumerate(ss):
    for t, pair in enumerate(sent):
        if len(sent) >= 2:
            wd, tag = sent[t][0],sent[t][1] 
    #         print (wd, tag)
            emp_sum_yx[tag_idx[tag],wd_idx[wd]] += 1
            if t < len(sent)-1:
                tag_next = sent[t+1][1]
                emp_sum_yy[tag_idx[tag_next],tag_idx[tag]] += 1

In [61]:
emp_sum_yx

array([[7.257e+03, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [0.000e+00, 1.700e+01, 8.500e+01, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.000e+00, 0.000e+00,
        0.000e+00],
       ...,
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00]])

### Forward:

$Z(x) = \Sigma_{y_T...y_1}\Pi_{t=1}^{T}\psi_t({y_t,y_{t-1},x_t})$<br>

$ = \Sigma_{y_T}\Sigma_{y_{T-1}}\psi_T(y_T,y_{T-1},x_T)\Sigma_{y_{T-2}}\psi_{T-1}(y_{T-1},y_{T-2},x_{T-1})...\Sigma_{y_{1}}\psi_{2}(y_{2},y_{1},x_{2})\psi_{1}(y_{1},y_{0},x_{1})$

if we compute it from right to left, we can define <br>$\alpha_t(j) := \Sigma_{y_{t-1}}\psi_t(y_t=j,y_{t-1},x_t)...\Sigma_{y_{1}}\psi_{2}(y_{2},y_{1},x_{2})\psi_{1}(y_{1},y_{0},x_{1})$<br>
and have recursion <br>$\alpha_t(j) = \Sigma_{y_{t-1}=i}\psi_{t}(y_{t}=j,y_{t-1}=i,x_{t})\alpha_{t-1}(i)$ <br>

Finally,
$Z(x) = \Sigma_{y_T = i}\alpha_{T}(i)$

In [ ]:
def compute_Z(weights, i):
    
    